# XGBoost

The goal of this notebook is to provide a quick demonstration of XGBoost in conjunction with "traditional" sklearn processes.

In [ ]:
import multiprocessing
import pandas as pd
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

import xgboost as xgb

We will play with the [eye movements dataset](https://www.openml.org/search?type=data&sort=runs&id=1044&status=active) from openml.

It contains various eye movements metrics measured as participants scan a list of news titles.

The setup is that participants are given a question, and then are presented 10 news titles. 5 are irrelevant to the question asked, 4 are relevant but do not answer the question, and 1 is relevant and answers the question.

The original data presents the eye movements metrics for each single words in the titles, for the sake of simplifying the data structure (which should normaly account for dependency between words in the same title and so on), we have aggregated the words by title for each assignment (we used a mean aggregation).


The target is encoded as:
 * 0 : title irrelevant
 * 1 : title relevant but does not answer the question
 * 2 : title relevant and answers the question

The features are:
 * **fixcount** Number of fixations to the word 
 * **firstPassCnt** Number of fixations to the word when it is first encountered 
 * **P1stFixation** '1' if fixation occured when the sentence the word was in was encountered the first time 
 * **P2stFixation** '1' if fixation occured when the sentence the word was in was encountered the second time 
 * **prevFixDur** Duration of previous fixation 
 * **firstfixDur** Duration of the first fixation when the word is first encountered 
 * **firstPassFixDur** Sum of durations of fixations when the word is first encountered 
 * **nextFixDur** Duration of the next fixation when gaze initially moves from the word 
 * **firstSaccLen** Length of the first saccade 
 * **lastSaccLen** Distance between fixation on the word and the next fixation 
 * **prevFixPos** Distance between the first fixation preceding the word and the beginning ot the word 
 * **landingPos** Distance between the first fixation on the word and the beginning of the word 
 * **leavingPos** Distance between the last fixation on the word and the beginning of the word 
 * **totalFixDur** Sum of all durations of fixations to the word 
 * **meanFixDur** Mean duration of the fixations to the word 
 * **nRegressFrom** Number of regressions leaving from the word 
 * **regressLen** Sum of durations of regressions initiating from this word 
 * **nextWordRegress** '1' if a regression initiated from the following word 
 * **regressDur** Sum of durations of the fixations on the word during regression 
 * **pupilDiamMax** Maximum pupil diameter 
 * **pupilDiamLag** Maximum pupil diameter 0.5 - 1.5 seconds after the beginning of fixation 
 * **timePrtctg** First fixation duration divided by the total number of fixations 


In [ ]:
eye_movements = pd.read_csv("../data/eye_movements_aggregated.csv")
eye_movements.head()

In [ ]:
eye_movements.shape

In [ ]:
eye_features = eye_movements.loc[:,['fixcount', 'firstPassCnt', 'P1stFixation', 'P2stFixation',
       'prevFixDur', 'firstfixDur', 'firstPassFixDur', 'nextFixDur',
       'firstSaccLen', 'lastSaccLen', 'prevFixPos', 'landingPos', 'leavingPos',
       'totalFixDur', 'meanFixDur', 'nRegressFrom', 'regressLen',
       'nextWordRegress', 'regressDur', 'pupilDiamMax', 'pupilDiamLag',
       'timePrtctg']]
labels = eye_movements['target'].astype(int)

# a simple XGBoost classifier

In [ ]:
from sklearn.model_selection import train_test_split

Xt ,Xv , yt, yv = train_test_split(eye_features , labels , stratify=labels)

print(f"train set length: {len(yt)}")
print(f"valid set length: {len(yv)}")

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count()-2)

xgb_model.fit( Xt, yt)

pd.crosstab( yv, xgb_model.predict(Xv) ,  rownames=['true'], colnames=['predicted'] )


We can integrate this classifier seemlessly in other sklearn routine such as gridSearchCV:

In [ ]:
%%time

xgb_model = xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count()-2)

# Make sure the number of threads is balanced.
clf = GridSearchCV(
    xgb_model,
    {"max_depth": [2, 4, 6], "n_estimators": [50, 100, 200], 'eta' : [0.01,0.1,1.0]},
    scoring='roc_auc_ovr',
    verbose=1,
    n_jobs=1,
)
clf.fit(Xt, yt)
print(clf.best_score_)
print(clf.best_params_)


# tree method and max bin number: 

[treemethod](https://xgboost.readthedocs.io/en/stable/treemethod.html)

exact , approx, hist, corresponding to different degrees of speed and performance.



 * `approx` or `hist` should generally be prefered
 * Most of the time using hist with higher `max_bin` can achieve similar or even superior accuracy while maintaining good performance.





In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
from time import time

## recording training time and performance gain
## for different values of max_bin

time_list = []
scores = []
mb_list = []

for mb in [2,4,8,16,32,64,128,256,512,1024]:

    xgb_model = xgb.XGBClassifier(
        n_jobs=multiprocessing.cpu_count(),
        tree_method="hist" , max_bin = mb )


    t1 = time()
    cv_score = cross_val_score(xgb_model , eye_features, labels , cv = 5, scoring='roc_auc_ovr')
    time_list.append( time()-t1 )
    
    scores.append( cv_score.mean() )
    mb_list.append( mb )

In [ ]:
fig,ax=plt.subplots(1,2,figsize = (10,5))
ax[0].scatter( mb_list , time_list )
ax[1].scatter( mb_list , scores )

# XGBoost number of estimators, eta,  and validation

In [ ]:
##let's start by splitting the data into a train and a validaiton set
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X_train, X_valid, y_train, y_valid = train_test_split(eye_features, labels , stratify=labels, random_state=94)

The XGB object lets us use a validation set to monitor the model during the different iterations

In [ ]:
xgb_model = xgb.XGBClassifier(tree_method="hist", 
                        eval_metric = "auc", n_estimators=10)

xgb_model.fit(X_train, y_train, eval_set=[(X_train, y_train),(X_valid, y_valid)] )

we can access this data in the model:

In [ ]:
xgb_model.evals_result_

In [ ]:
plt.plot( xgb_model.evals_result_['validation_0']['auc'] , label='train')
plt.plot( xgb_model.evals_result_['validation_1']['auc'] , label='validation')
plt.legend()

After some point we reach a plateau, and then we start over-fitting the train data:

In [ ]:
%%time
xgb_model = xgb.XGBClassifier(
    n_jobs=multiprocessing.cpu_count()-2,
    eta = 0.3 , n_estimators=100 , 
    eval_metric= 'auc' )

xgb_model.fit(X_train , y_train  , 
              eval_set=[(X_train, y_train),(X_valid, y_valid)],
              verbose=False )
    
plt.plot( xgb_model.evals_result_['validation_0']['auc'] , label='train')
plt.plot( xgb_model.evals_result_['validation_1']['auc'] , label='validation')
plt.legend()

`eta` controls how fast we learn.

 * lower value: slower learning
 * higher value: faster learning, but potentially unstable

In [ ]:
%%time
xgb_model = xgb.XGBClassifier(
    n_jobs=multiprocessing.cpu_count()-2,
    tree_method="hist" , eta = 0.05 , n_estimators=100 , 
    eval_metric= 'auc' )

xgb_model.fit(X_train , y_train  , 
              eval_set=[(X_train, y_train),(X_valid, y_valid)],
              verbose=False )
    
plt.plot( xgb_model.evals_result_['validation_0']['auc'] , label='train')
plt.plot( xgb_model.evals_result_['validation_1']['auc'] , label='validation')
plt.legend()

In [ ]:
%%time
xgb_model = xgb.XGBClassifier(
    n_jobs=multiprocessing.cpu_count()-2,
    tree_method="hist" , eta = 0.5 , n_estimators=100 , 
    eval_metric= 'auc' )

xgb_model.fit(X_train , y_train  , 
              eval_set=[(X_train, y_train),(X_valid, y_valid)],
              verbose=False )
    
plt.plot( xgb_model.evals_result_['validation_0']['auc'] , label='train')
plt.plot( xgb_model.evals_result_['validation_1']['auc'] , label='validation')
plt.legend()

Then of course we may want to stop the iterations when we observe a decrease of the validation performance: that's **early stopping**

In [ ]:
## early stopping

clf = xgb.XGBClassifier(tree_method="hist", 
                        early_stopping_rounds=5 , 
                        eval_metric = "auc",
                        eta = 0.05 )
clf.fit(X_train, y_train, eval_set=[(X_train, y_train),(X_valid, y_valid)])

In [ ]:
clf.best_iteration , clf.best_score #  These are used by the predict() 

Early stopping is great to balance the number of estimators and eta, but, as noted in the [XGBoost doc](https://xgboost.readthedocs.io/en/stable/python/sklearn_estimator.html#early-stopping):

"""
 However, using **early stopping during cross validation may not be a perfect approach because it changes the model’s number of trees for each validation fold, leading to different model.** A better approach is to retrain the model after cross validation using the best hyperparameters along with early stopping. If you want to experiment with idea of using cross validation with early stopping, here is a snippet to begin with:. However, using early stopping during cross validation may not be a perfect approach because it changes the model’s number of trees for each validation fold, leading to different model. A better approach is to retrain the model after cross validation using the best hyperparameters along with early stopping. 
 
 """


We can use it here to evaluate the effect of `eta`:

In [ ]:
import numpy as np

In [ ]:
%%time
best_scores = []
num_rounds = []
etas = np.logspace(-3,0,50)
for eta in etas:
    xgb_model = xgb.XGBClassifier(
        n_jobs=multiprocessing.cpu_count()-2,
        early_stopping_rounds=10 ,
        eta = eta , n_estimators=500 , 
        eval_metric= 'auc' )

    xgb_model.fit(X_train , y_train  , 
                  eval_set=[(X_train, y_train),(X_valid, y_valid)],
                  verbose=False )

    best_scores.append( xgb_model.best_score )
    num_rounds.append( xgb_model.best_iteration )

fig,ax=plt.subplots(1,2,figsize = (10,5))
ax[0].plot( etas , best_scores )
ax[0].set_xlabel('eta')
ax[0].set_ylabel('score')
ax[0].set_xscale('log')
ax[0].grid()

ax[1].plot( etas , num_rounds )
ax[1].set_xlabel('eta')
ax[1].set_ylabel('number of rounds')
ax[1].set_xscale('log')
ax[1].grid()


## exercise: 

Test the effect of the `subsample` parameter on XGboost. This argument can take values from 0.0 to 1.0.

In [ ]:
# %load solutions/solution_xgb_subsample.py

# (some of) XGBoost hyper-parameters to control overfitting

from the [recommendations on controling overfitting from the documentation](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html#control-overfitting)



 * The first way is to directly control model complexity.
     * This includes `max_depth`, `min_child_weight` and `gamma`.

 * The second way is to add randomness to make training robust to noise.
     * This includes `subsample` and `colsample_bytree`.
     * You can also reduce stepsize `eta`. Remember to increase `num_round` when you do so.


We have already looked at `eta` and `subsample` together.


## max_depth

max_depth (Optional[int]) – Maximum tree depth for base learners.

In [ ]:
# let's make a helper function to facilitate our work
def test_param( param_name, param_values, **kwargs ):

    best_scores_val = []
    best_scores_train = []
  
    for value in param_values:
        xgb_model = xgb.XGBClassifier(
            n_jobs=multiprocessing.cpu_count()-2,
            early_stopping_rounds=10 ,
            eval_metric= 'auc',
            **kwargs,
            **{param_name:value} 
        )

        xgb_model.fit(X_train , y_train  , 
                      eval_set=[(X_train, y_train),(X_valid, y_valid)],
                      verbose=False )

        best_scores_val.append( xgb_model.best_score )
        best_scores_train.append( xgb_model.evals_result_['validation_0']['auc'][xgb_model.best_iteration] )
        


    return param_values , best_scores_val , best_scores_train

In [ ]:
eta=0.1
n_estimators = 100

In [ ]:
pname = 'max_depth'
pvals = np.arange(1,20)

param_values , best_scores_val , best_scores_train = test_param( pname, pvals , 
                                                                eta=eta , n_estimators=n_estimators)
fig,ax=plt.subplots(1,1,figsize = (10,5))
ax.plot( param_values , best_scores_val , label = 'validation set' )
ax.plot( param_values , best_scores_train , label = 'train set' )
ax.set_xticks( param_values )
ax.set_xlabel(pname)
ax.set_ylabel('score')
ax.grid()
ax.legend()

## gamma

gamma (Optional[float]) – (min_split_loss) Minimum loss reduction required to make a further partition on a leaf node of the tree.

In [ ]:
pname = 'gamma'
pvals = np.linspace(0.0,40,100)

param_values , best_scores_val , best_scores_train = test_param( pname, pvals,
                                                               eta=eta , n_estimators=n_estimators)
fig,ax=plt.subplots(1,1,figsize = (10,5))
ax.plot( param_values , best_scores_val , label = 'validation set' )
ax.plot( param_values , best_scores_train , label = 'train set' )
#ax.set_xticks( param_values )
ax.set_xlabel(pname)
ax.set_ylabel('score')
ax.grid()
ax.legend()

## min_child_weight

min_child_weight (Optional[float]) – Minimum sum of instance weight(hessian) needed in a child.

In [ ]:
pname = 'min_child_weight'
pvals = np.linspace(0.0,200,50)

param_values , best_scores_val , best_scores_train = test_param( pname, pvals,
                                                               eta=eta , n_estimators=n_estimators)
fig,ax=plt.subplots(1,1,figsize = (10,5))
ax.plot( param_values , best_scores_val , label = 'validation set' )
ax.plot( param_values , best_scores_train , label = 'train set' )
#ax.set_xticks( param_values )
ax.set_xlabel(pname)
ax.set_ylabel('score')
ax.grid()
ax.legend()

## colsample_bytree

colsample_bytree (Optional[float]) – Subsample ratio of columns when constructing each tree.

In [ ]:
pname = 'colsample_bytree'
pvals = np.linspace(0.1,1.0,50)

param_values , best_scores_val , best_scores_train = test_param( pname, pvals ,
                                                               eta=eta , n_estimators=n_estimators)
fig,ax=plt.subplots(1,1,figsize = (10,5))
ax.plot( param_values , best_scores_val , label = 'validation set' )
ax.plot( param_values , best_scores_train , label = 'train set' )
#ax.set_xticks( param_values )
ax.set_xlabel(pname)
ax.set_ylabel('score')
ax.grid()
ax.legend()

And these are just some of the main hyper-parameters.

**micro-exercise:**
what may be missing from the hyper-parameter explorations we have done here?


<br>

<br>

<br>

---


# Annex : 

I put here some additional remarks and tricks

## Imbalance

https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html#handle-imbalanced-dataset

the documentation recommends to use `scale_pos_weight`, which corresponds to the re-weigthing of the positive class samples

typical value num_negative_samples / num_positive_samples

Let's try it out:

In [ ]:
from sklearn.datasets import make_classification

B = 0.99
X , y = make_classification(n_samples=3*10**3 , weights=[B,1-B])

Xt, Xv, yt, yv = train_test_split(X, y, stratify=y, random_state=89)

num_pos = yt.sum()

xgb_model = xgb.XGBClassifier(
    n_jobs=multiprocessing.cpu_count()-2, 
    early_stopping_rounds=10 ,
    eval_metric= 'auc',
    scale_pos_weight =  (len(yt) - num_pos) / num_pos )


xgb_model.fit(Xt , yt  , 
                  eval_set=[(Xt, yt),(Xv, yv)],
                  verbose=False )
print( xgb_model.best_score )

pd.crosstab( yv,  xgb_model.predict( Xv ) ,  rownames=['true'], colnames=['predicted'] )

In [ ]:
%%time
from sklearn.datasets import make_classification
prop_zeros = np.logspace( np.log10(0.5) , np.log10(1-10**-3) , 50 )
auc_no_weights = []
auc_with_weights = []
recall_with_weight = []
recall_no_weight = []

for B in prop_zeros:
    X , y = make_classification(n_samples=3*10**3 , weights=[B,1-B])

    Xt, Xv, yt, yv = train_test_split(X, y, stratify=y, random_state=89)

    num_pos = yt.sum()

    ## with weights
    xgb_model = xgb.XGBClassifier(
        n_jobs=multiprocessing.cpu_count()-2, 
        early_stopping_rounds=10 ,
        eval_metric= 'auc',
        scale_pos_weight = (len(yt) - num_pos) / num_pos )


    xgb_model.fit(Xt , yt  , 
                      eval_set=[(Xt, yt),(Xv, yv)],
                      verbose=False )
    auc_with_weights.append( xgb_model.best_score )
    recall_with_weight.append( ( xgb_model.predict( Xv )[yv==1] ==1 ).mean() )
    ## without weights
    xgb_model = xgb.XGBClassifier(
        n_jobs=multiprocessing.cpu_count()-2, 
        early_stopping_rounds=10 ,
        eval_metric= 'auc')


    xgb_model.fit(Xt , yt  , 
                      eval_set=[(Xt, yt),(Xv, yv)],
                      verbose=False )
    auc_no_weights.append( xgb_model.best_score )
    
    recall_no_weight.append( ( xgb_model.predict( Xv )[yv==1] ==1 ).mean() )


In [ ]:
plt.plot( prop_zeros , auc_no_weights , label="no re-weigthing" )
plt.plot( prop_zeros , auc_with_weights , label="re-weigthing" )
plt.xlabel('imbalance')
plt.ylabel('AUC')
plt.legend()

In [ ]:
plt.plot( prop_zeros , recall_no_weight , label="no re-weigthing" )
plt.plot( prop_zeros , recall_with_weight , label="re-weigthing" )
plt.xlabel('imbalance')
plt.ylabel('recall')
plt.legend()

The effect is not extremely clear here, but recall tends to be better.

## trick: continuing training 

In [ ]:
xgb_model = xgb.XGBClassifier(
    n_jobs=multiprocessing.cpu_count()-2,
    tree_method="hist" , eta = 0.3 , n_estimators=32  )

xgb_model.fit(X_train , y_train)

print( "first call to fit:", xgb_model.get_booster().num_boosted_rounds() , 'rounds')

xgb_model.fit(X_train , y_train  ,xgb_model=xgb_model )

print( "second call to fit:", xgb_model.get_booster().num_boosted_rounds() , 'rounds')